In [2]:
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from datetime import datetime, timedelta


default_args = dict(
    owner='airflow',
    depends_on_past=False,
    start_date=datetime(2018, 1, 1),
    email=['f.j.akimoto@gmail.com'],
    email_on_failure=False,
    email_on_retry=False,
    retries=1,
    retry_delay=timedelta(minutes=5))

In [3]:
dag = DAG('tutorial', default_args=default_args)

t1 = BashOperator(
    task_id='print_date',
    bash_commands='date',
    dag=dag)

t2 = BashOperator(
    task_id='sleep',
    bash_commands='sleep 5',
    retries=3,
    dag=dag)

templated_command = """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7) }}"
        echo "{{ params.my_param }}"
    {% endfor %}
"""

t3 = BashOperator(
    task_id='templated',
    bash_commands=templated_command,
    params={'my_param': 'Parameter I passed in'},
    dag=dag)

t2.set_upstream(t1)
t3.set_upstream(t2)

AirflowException: Argument ['bash_command'] is required

In [4]:
def get_data(pair, data_dir, start, end, period=1800):
    datafile = os.path.join(data_dir, pair + ".csv")
    timefile = os.path.join(data_dir, pair)

    start_sec = date2seconds(start)
    end_sec = date2seconds(end)

    FETCH_URL = "https://poloniex.com/public?command=returnChartData&currencyPair=%s&start=%d&end=%d&period=%d"
    COLUMNS = ["date", "high", "low", "open",
               "close", "volume", "quoteVolume", "weightedAverage"]

    url = FETCH_URL % (pair, start_sec, end_sec, period)
    print("Get %s from %d to %d with period %d" % (pair, start_sec, end_sec, period))

    df = pd.read_json(url, convert_dates=False)

    if df["date"].iloc[-1] == 0:
        print("No data.")
        return

    end_time = df["date"].iloc[-1]
    ft = open(timefile, "w")
    ft.write("%d\n" % end_time)
    ft.close()
    outf = open(datafile, "w")
    df.to_csv(outf, index=False, columns=COLUMNS)
    outf.close()
    print("Finish.")
    time.sleep(5)

In [48]:
from datetime import datetime
import time

def date2seconds(str_time):
    datetime_obj = datetime.strptime(str_time, '%Y-%m-%d %H:%M:%S')
    seconds = time.mktime(datetime_obj.timetuple())
    return seconds


def seconds2date(seconds):
    date_obj = time.localtime(seconds)
    str_time = '%04d-%02d-%02d %02d:%02d:%02d'
    return str_time % (date_obj.tm_year, date_obj.tm_mon, date_obj.tm_mday,
                       date_obj.tm_hour, date_obj.tm_min, date_obj.tm_sec) 

In [49]:
# from rltensor.environments.utils import date2seconds

end = '2016-09-07 04:00:00'
sec = date2seconds(end)
print(sec)
d = seconds2date(sec)
print(d)

1473235200.0
2016-09-07 04:00:00


In [35]:
sec = date2seconds(d)
print(sec)
d = seconds2date(sec)
print(d)

1473580800.0
2016-09-11 08:00:00


In [6]:
t1

time.struct_time(tm_year=2010, tm_mon=9, tm_mday=12, tm_hour=10, tm_min=19, tm_sec=54, tm_wday=6, tm_yday=255, tm_isdst=0)

In [9]:
'%02d' % 12

'12'